<a href="https://colab.research.google.com/github/joseverajim/Actividad-3.1-3.2-y-3.3-Valores-Nulos-/blob/main/Actividad3.2/Josevera/Actividad_3_2_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np #
import re

# 1) Cargar el archivo Excel y la hoja 2023
file_path = "Gastos y costos 20-23.xlsx"   # Asegúrate que esté en la misma carpeta
df2023 = pd.read_excel(file_path, sheet_name="2023", header=3)

In [3]:
#Revisar columnas del dataframe
df2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Fecha        397 non-null    datetime64[ns]
 1   Folio        384 non-null    object        
 2   UUID         397 non-null    object        
 3   RFC          397 non-null    object        
 4   Proveedor    397 non-null    object        
 5   TIPO GASTO   389 non-null    object        
 6   Descripción  397 non-null    object        
 7   MP           326 non-null    object        
 8   FP           326 non-null    float64       
 9   TC           397 non-null    float64       
 10  Importe      397 non-null    float64       
 11  IVA          397 non-null    float64       
 12  RET ISR      397 non-null    float64       
 13  RET IVA      397 non-null    float64       
 14  Otros        0 non-null      float64       
 15  TOTAL MX     397 non-null    float64       
 16  TOTAL SA

In [5]:
#Revisar cuántos vacíos hay por columna
print(df2023.isnull().sum())

Fecha            0
Folio           13
UUID             0
RFC              0
Proveedor        0
TIPO GASTO       8
Descripción      0
MP              71
FP              71
TC               0
Importe          0
IVA              0
RET ISR          0
RET IVA          0
Otros          397
TOTAL MX         0
TOTAL SAT        0
Tipo             0
Status           0
Poliza         115
dtype: int64


In [6]:
# Revisar qué pasa con los registros que tienen Folio nulo
print(df2023[df2023["Folio"].isnull()].head(20))

         Fecha Folio                                  UUID            RFC  \
17  2023-01-01   NaN  8BCDE3BB-6B66-4EB9-9E68-73622875994F   TME840315KT6   
90  2023-01-01   NaN  B9943156-B044-4FC9-BA4F-7E3EDE49AF12  SOMA690808RT6   
93  2023-01-01   NaN  30B89A77-37E1-4AC8-996B-8F8B303E7A96  LAFC670511E51   
125 2023-01-01   NaN  14F135F6-1469-493D-938C-1D536747FF65   NFM0307091L9   
191 2023-01-01   NaN  03A1E420-0D87-4DBA-818B-EDEF6D7A1E65   TME840315KT6   
244 2023-02-01   NaN  E8E20C44-7C1E-4824-87F2-CEE040A0FFD8  SAHM881103BR1   
257 2023-02-01   NaN  8419C591-AE98-44D1-9298-7362287591E7   TME840315KT6   
262 2023-02-01   NaN  B2301ACC-DC39-4F46-BBB3-C59C7501D356  SAAJ7708108H7   
327 2023-02-01   NaN  91223FBB-1CD9-41EC-B0BA-A9D7097D1378  SOMA690808RT6   
338 2023-02-01   NaN  643862D4-D79D-4B17-A31F-BE73C6396A87   NFM0307091L9   
343 2023-02-01   NaN  F943E10C-1C49-4DAE-B911-4EF2CE95FB45  LAFC670511E51   
355 2023-02-01   NaN  68626F73-33DF-41B3-AC86-73622875B6C7   TME840315KT6   

In [7]:
# 1. Sustitución de valores de  Folio
# Este bloque identifica las filas donde el Folio está vacío (mask = True).
# Luego genera valores únicos de reemplazo: "SIN_FOLIO_1", "SIN_FOLIO_2", ...
# De esta manera, cada registro conserva un identificador aunque no tenga Folio original, evitando duplicados o pérdida de información y podiendo identificar cuales fueron los datos imputados.
mask = df2023["Folio"].isnull()
df2023.loc[mask, "Folio"] = [f"SIN_FOLIO_{i}" for i in range(1, mask.sum()+1)]

In [8]:
# 2. Eliminamos las columnas de MP Y FP vacias
# --- Comprobación específica de MP y FP ---
# Sabemos por la revisión en Excel que los valores faltantes en estas columnas aparecen como celdas vacías o con el símbolo "-".
# Hacemos una verificación rápida de cuántos registros cumplen esa condición.

mp_invalidos = ((df2023["MP"].isna()) | (df2023["MP"] == "-")).sum()
fp_invalidos = ((df2023["FP"].isna()) | (df2023["FP"] == "-")).sum()

print(f"Registros inválidos detectados en MP: {mp_invalidos}")
print(f"Registros inválidos detectados en FP: {fp_invalidos}")

Registros inválidos detectados en MP: 71
Registros inválidos detectados en FP: 71


In [9]:
# --- Eliminación de esas filas ---
# Eliminamos los registros que tengan valores vacíos o "-" en MP o FP
df2023 = df2023[~(
    (df2023["MP"].isna()) | (df2023["MP"] == "-") |
    (df2023["FP"].isna()) | (df2023["FP"] == "-")
)]

In [10]:
# 3.- Eliminación de la columna 'Otros'
# --- Comprobación de la columna 'Otros' ---
# Según lo observado en Excel, toda la columna está vacía.
# Verificamos cuántos valores nulos hay y el total de registros.

nulos_otros = df2023["Otros"].isna().sum()
total_registros = len(df2023)

print(f"Nulos en 'Otros': {nulos_otros} de {total_registros}")


Nulos en 'Otros': 326 de 326


In [11]:
# --- Eliminación de la columna ---
# Como 'Otros' está completamente vacía, se elimina del DataFrame.
df2023 = df2023.drop(columns=["Otros"])

In [12]:
# 4. Sustitución de valores de Poliza
# Identificamos las filas donde Poliza está vacío (mask = True).
# Luego generamos valores únicos de reemplazo: "SIN_POLIZA_1", "SIN_POLIZA_2", ...
# Así cada registro conserva un identificador único, evitando duplicados y dejando claro cuáles fueron imputados.

mask = df2023["Poliza"].isnull()
df2023.loc[mask, "Poliza"] = [f"SIN_POLIZA_{i}" for i in range(1, mask.sum()+1)]


In [13]:
# 5. Imputación de TIPO GASTO
# Obtener la moda de TIPO GASTO
moda_tipo_gasto = df2023["TIPO GASTO"].mode()[0]

# Sustituir nulos con la moda
df2023["TIPO GASTO"] = df2023["TIPO GASTO"].fillna(moda_tipo_gasto)

In [14]:
# Comprobación de que ya no queden nulos
print("Nulos restantes por columna:")
print(df2023.isnull().sum())

# Mostrar algunos ejemplos de Folio y Poliza imputados
print("\nEjemplos Folio:")
print(df2023["Folio"].head(10))

print("\nEjemplos Poliza:")
print(df2023["Poliza"].head(10))

# Mostrar la moda utilizada y algunos ejemplos de TIPO GASTO
moda_tipo_gasto = df2023["TIPO GASTO"].mode()[0]
print(f"\nModa utilizada para TIPO GASTO: {moda_tipo_gasto}")
print("Ejemplos TIPO GASTO:")
print(df2023["TIPO GASTO"].head(10))

Nulos restantes por columna:
Fecha          0
Folio          0
UUID           0
RFC            0
Proveedor      0
TIPO GASTO     0
Descripción    0
MP             0
FP             0
TC             0
Importe        0
IVA            0
RET ISR        0
RET IVA        0
TOTAL MX       0
TOTAL SAT      0
Tipo           0
Status         0
Poliza         0
dtype: int64

Ejemplos Folio:
2                     341932
3                     341933
4                     342205
5     0001010100750847116902
6               007817297284
7                         56
8                   90864603
14                   4640966
15            22022120028311
17               SIN_FOLIO_1
Name: Folio, dtype: object

Ejemplos Poliza:
2       PE01-ENE23
3       PE02-ENE23
4       PE03-ENE23
5       PE50-FEB23
6     SIN_POLIZA_1
7      PE100-ENE23
8      PE136-ENE23
14    SIN_POLIZA_2
15    SIN_POLIZA_3
17      PE96-ENE23
Name: Poliza, dtype: object

Moda utilizada para TIPO GASTO: COMPRAS
Ejemplos TIPO GASTO:
2  

In [15]:
# Guardar DataFrame limpio en CSV
df2023.to_csv("df2023_limpio.csv", index=False, encoding="utf-8-sig")

print("Archivo guardado como: df2023_limpio.csv")


Archivo guardado como: df2023_limpio.csv
